<font size="5">

# Multi-Grid measurements at LET, 2022
    
  

> __Author:__ A. Backis
<br/>__Institute:__ University of Glasgow (UoG), European Spallation Source (ESS)
    
> __Author:__ R. Wahlén
<br/>__Institute:__ Lund University (LU), European Spallation Source (ESS)

>__Date:__ 7/4-2022
    
_Abstract:_
This notebook contains the data-analysis tools used for the measurements at the LET instrument at ISIS. It describes how the analysis was performed, and summarizes the results.

# Contents
    
* [1. Introduction](#INTRODUCTION)
    * [1.1 Packages](#PACKAGES)
    * [1.2 Parameters](#PARAMETERS)
* [2. STF](#STF)

# 1. Introduction<a class="anchor" id="INTRODUCTION"></a>

Notebook showing the analysis of the LET measurements.

## 1.1 Packages<a class="anchor" id="PACKAGES"></a>

Import required packages.

In [ ]:
# Autoload packages when doing an external change
%load_ext autoreload
%autoreload 2

# Activate matplotlib in interactive notebook mode
%matplotlib widget

# General packages
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure custom packages can be reached
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Local packages
import file_handling.he3_read as he3_read
import file_handling.bm_read as bm_read
import file_handling.mg_ref.mg_ref_read as mg_read
import file_handling.mg_ref.mg_ref_manage as mg_manage

import plotting.mg_ref_basic_plot as mg_basic_plot
import plotting.he3_basic_plot as he3_basic_plot
import plotting.bm_basic_plot as bm_basic_plot
import plotting.lineshape as lineshape
import plotting.common_plot as cmnplt

## 1.2 Parameters<a class="anchor" id="PARAMETERS"></a>

Define global parameters.

In [ ]:
# Instrument definition
MODERATOR_TO_SAMPLE_IN_M = 25

# Paths to folders containing data
nb_path = os.getcwd()
MG_RAW_FOLDER = nb_path + '/../data/mg/ref/raw/'
MG_PROCESSED_FOLDER = nb_path + '/../data/mg/ref/processed/'
HE3_FOLDER = nb_path + '/../data/he3/'

# Filter for Multi-Grid detector
mg_filter = {'wm': [1, 1, False],                   # Wire multiplicity
             'gm': [1, 5, False],                   # Grid multiplicity
             'wadc': [600, np.inf, False],          # Wire charge
             'gadc': [600, np.inf, False],          # Grid charge
             'tof': [0, np.inf, False],             # Time-of-flight (TDC channels)
             'time': [0, np.inf, False],            # Time (TDC channels)
             'bus': [2, 3, False],                  # Bus
             'layer': [0, 19, False],              # Layer, front=0 to back=19
             'row': [0, 11, False],                # Row, right to left (seen from neutrons)
             'gch': [81, 118, False]}               # Grid channel, bottom=96 to top=132

# Filter for Helium-3 tubes
pixels = []
start_pixel = 38400+1 # As the Multi-Grid detector is elevated a few centimeters from the ground
height = 58
diff_tubes = 256
number_tubes = 32
number_rows = 4
for i in np.arange(0, number_rows, 1):
    start = start_pixel+diff_tubes*i
    stop = start_pixel+diff_tubes*i + height
    for j in np.arange(start, stop):
        pixels.append(j)
region_of_interest = np.array(pixels)

region_edges = np.zeros(5, dtype='int')
region_edges[0] = region_of_interest[0]
region_edges[1] = region_of_interest[height-1]
region_edges[2] = region_of_interest[number_rows*height-1]
region_edges[3] = region_of_interest[number_rows*height-height]
region_edges[4] = region_of_interest[0]

## 2. STF<a class="anchor" id="STF"></a>

Test data set taken with an AmBe source in the source-testing facility.

In [ ]:
# Declare file name
file_name = 'CSPEC-LET_STF_Test_220405_165718_Th6'
raw_path = MG_RAW_FOLDER + file_name + '.zip'
# Declare area
mg_area = 0.025*0.025*6*37
# Extract and save (only has to be done once)
#mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Load data
#clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Filter data
#clu_f = mg_manage.filter_data(clu, mg_filter)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter, mg_area, file_name)

## 3. Mapping test<a class="anchor" id="MAPPING_TEST"></a>

In [ ]:
# Helium-3 data
he3_file_name = 'LET00068153'
he3_path = HE3_FOLDER + he3_file_name + '.nxs'
he3_mapping = he3_read.get_pixel_to_xyz_mapping(he3_path)
df_he3_run = he3_read.import_data(he3_path)
# Multi-Grid data
mg_file_name = 'CSPEC-LET_STF_Test_220405_165718_Th6'
df_mg, __ = mg_manage.load_clusters_and_events(mg_file_name, MG_PROCESSED_FOLDER)
# Plot Helium-3 histogram
he3_basic_plot.he3_plot_3D_histogram(df_he3_run, he3_mapping, region_edges=region_edges)
# Plot Multi-Grid histogram
mg_basic_plot.clusters_3d_plot(clu[clu.bus == 9], file_name)
# Plot Helium-3 + Multi-Grid
mg_offset = {'x': -1.5, 'y': -1, 'z': 2.9}
mg_theta = -60 * np.pi/180
cmnplt.plot_mg_and_he3_3d(df_mg[df_mg.bus == 9], df_he3_run, he3_mapping, mg_offset, mg_theta, region_edges=region_edges)